# 🏠 Ev Fiyat Tahmin Projesi - Modelleme ve Değerlendirme

Bu notebook, temel bir LightGBM modeli ile hızlı değerlendirme içerir (sadece sayısal özelliklerle).


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor

pd.set_option('display.max_columns', None)
print("✅ Kütüphaneler yüklendi")


In [ ]:
# İşlenmiş veri üzerinden modelleme
# Önce 2. notebook'u çalıştırıp processed_data.csv üretildiğinden emin olun

df = pd.read_csv('processed_data.csv')
train_df = df[df['SalePrice'].notnull()].copy()
test_df = df[df['SalePrice'].isnull()].copy()

y = train_df['SalePrice']
X = train_df.drop(columns=['Id','SalePrice'], errors='ignore')

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_val.shape


In [ ]:
# Basit LightGBM modeli
model = LGBMRegressor(random_state=42)
model.fit(X_train, y_train)

pred = model.predict(X_val)
rmse = (mean_squared_error(y_val, pred))**0.5
print(f"RMSE: {rmse:,.2f}")


> Not: Bu notebook hızlı bir doğrulama sunar. Tam pipeline için veri ön işleme notebook'una bakınız.


## 🔧 Hiperparametre Optimizasyonu (Hızlı Grid)
Daha iyi sonuç için küçük bir grid ile deneme yapılır.


In [ ]:
from sklearn.model_selection import GridSearchCV
params = {
    'learning_rate':[0.01, 0.1],
    'n_estimators':[500, 1500],
}
gs = GridSearchCV(LGBMRegressor(random_state=42), params, cv=3,
                  scoring='neg_mean_squared_error', n_jobs=-1)
gs.fit(X_train, y_train)
print('Best params:', gs.best_params_)
print('CV RMSE:', (-gs.best_score_)**0.5)


## 📈 Feature Importance
En önemli değişkenleri görselleştirelim.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
best_model = gs.best_estimator_
best_model.fit(X_train, y_train)
imp = best_model.feature_importances_
idx = np.argsort(imp)[-20:]
plt.figure(figsize=(8,6))
plt.barh(range(len(idx)), imp[idx])
plt.yticks(range(len(idx)), X_train.columns[idx])
plt.title('Feature Importance (Top 20)')
plt.tight_layout()
plt.show()


## 🔁 Çoklu Model Karşılaştırma (CV RMSE)
Script'teki model listesini kullanarak 5-katlı CV ile RMSE karşılaştırması.


In [ ]:
models = [
    ('LR', LinearRegression()),
    ('KNN', KNeighborsRegressor()),
    ('CART', DecisionTreeRegressor()),
    ('RF', RandomForestRegressor()),
    ('GBM', GradientBoostingRegressor()),
    ('XGBoost', XGBRegressor(objective='reg:squarederror')),
    ('LightGBM', LGBMRegressor()),
]
for name, reg in models:
    rmse = np.mean(np.sqrt(-cross_val_score(reg, X, y, cv=5, scoring='neg_mean_squared_error')))
    print(f"RMSE: {rmse:.4f} ({name})")


## log1p Dönüşümü ile Modelleme
Log dönüşümü sonrası RMSE'yi inverse alarak hesaplayalım.


In [ ]:
y_log = np.log1p(train_df['SalePrice'])
X_full = train_df.drop(columns=['Id','SalePrice'], errors='ignore')
X_train, X_val, y_train, y_val = train_test_split(X_full, y_log, test_size=0.2, random_state=42)
log_model = LGBMRegressor(random_state=42).fit(X_train, y_train)
y_pred = log_model.predict(X_val)
rmse = np.sqrt(mean_squared_error(np.expm1(y_val), np.expm1(y_pred)))
print(f'Log-transform RMSE: {rmse:,.2f}')


## 📤 Kaggle Submission
Test seti için tahmin üretip `housePricePredictions.csv` olarak kaydedelim.


In [ ]:
final_model = LGBMRegressor(random_state=42)
final_model.fit(X_full, y_log)
X_test_final = test_df.drop(columns=['Id','SalePrice'], errors='ignore')
preds_log = final_model.predict(X_test_final)
preds = np.expm1(preds_log)
submission = pd.DataFrame({'Id': test_df['Id'].fillna(test_df.index), 'SalePrice': preds})
submission.to_csv('housePricePredictions.csv', index=False)
print('housePricePredictions.csv kaydedildi')
